In [1]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

In [2]:
def read_stop_words(file):
    with open(file) as f:
        stop_words = f.read().split('\n')

    return stop_words

In [5]:
!ls dataset/booking

analyze.ipynb
booking-rating-for-one-hot-test.csv
booking-rating-for-one-hot-train.csv
booking-rating-for-one-hot-val.csv
booking-rating-test.csv
booking-rating-train.csv
booking-rating-val.csv
booking-sentences-test.csv
booking-sentences-train.csv
booking-sentences-val.csv
booking-test.csv
booking-train.csv
booking-val.csv
create-dataset-for-rating-classification.ipynb
create-sentence-classification-dataset.ipynb
dnipro-reviews.csv
ivano-frankivsk-reviews.csv
kharkiv-reviews.csv
kyiv-reviews.csv
lviv-reviews.csv
odesa-reviews.csv
translated
uzhgorod-reviews.csv


In [6]:
def read_all_frames(files):
    frames= []
    for file in files:
        df = pd.read_csv(file)
        frames.append(df)
    
    return pd.concat(frames)

In [38]:
def is_review_valid(review):
    if type(review) is not str:
        return False
    try:
        if len(review) == 0:
            return False
        if review == 'Nan':
            return False
        return True
    except:
        return False

In [27]:
reviews

['Лише дівчата на рецепції - три рази мені міняли кімнату. Дякую їм.',
 'Все. Одного досвіду вистарчило, щоб більше сюди не повертатися. Готель лише для гостей, яким байдуже умови , чистота і комфорт.',
 'Оформлення кімнати хороше, досить приємне, на одну-дві ночі - чудовий варіант, особливо якщо врахувати ціну. Такі номери у конкурентів в два рази дорожчі.',
 'Nan',
 "Усе відмінно, завдяки якісному сервісу ми завжди тут зупиняємося. Цього разу ресторан було заброньовано, нам люб'язно запропонували вечерю в номер, це дуже зручно. Надзвичайно люб'язний персонал. Є територія що замикається, для мешканців з авто.",
 'Рекомендую людям котрі подорожують власним транспортом.',
 'Ціна/якість в принципі',
 'душова, трохи старий ремонт, але за 500 грн за щастя . Особливо не сподобалась басовито музика, як в 16 прибув, так і до 23 ї рубала.',
 'Приїхали з сином біля сьомої ранку, сонні та втомлені, нас заселили відразу ж . Дуже вдячна . Все чисто, гарно, свіжий ремонт, подушечки . З вікна ялиноч

In [8]:
full_df = read_all_frames(files)

In [16]:
len(full_df['pos_text'].values)

134083

In [39]:
def get_pos_neg_review(df):
    pos_texts = df['pos_text'].values
    neg_texts = df['neg_text'].values
    
    gt_score, reviews = [], []
    for i in range(0, len(df)):
        if is_review_valid(pos_texts[i]):
            reviews.append(pos_texts[i])
            gt_score.append(1)
            
        if is_review_valid(neg_texts[i]):
            reviews.append(neg_texts[i])
            gt_score.append(0)
            
    return reviews, gt_score

In [40]:
reviews, gt_score = get_pos_neg_review(full_df)

In [42]:
uk_stop_words = read_stop_words('./data/ukrainian-stopwords.txt')

### 1. Word frequency

In [43]:
def get_topk_ngram(df, ngram_range=(1,1), k=None, stopwords=True, with_count=False):
    '''
    Extract the most frequently occurred words in countvector
    '''
    if stopwords:
        temp = []
        for name in hotelDf.hotelName.unique():
            for token in name.split():
                if len(token) > 1:
                    temp.append(token)
        my_stop_words = ENGLISH_STOP_WORDS.union(temp)
        vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words=my_stop_words, max_features=500)
        
    else:
        vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words=None, max_features=k)
        
    countvector = vectorizer.fit_transform(df['review'])

    # Get topk occurred ngrams
    topk_words = []
    sortedindices = countvector.toarray().sum(axis=0).argsort()[::-1][:k]
    counts = countvector.toarray().sum(axis=0)
    
    for i in sortedindices:
        word = vectorizer.get_feature_names()[i]
        
        if with_count:
            count = counts[i]
            topk_words.append((word, count))
        else:
            topk_words.append(word)
            
    return topk_words

In [ ]:
topkTotal = get_topk_ngram(hotelDf, k=500)

### 2. Mutual information

**Mutual information tells you how much you learn about X from knowing the value of Y (on average over the choice of Y).** 



Since we found the word frequency is not a good indicator for the sentiment analysis, we will examine *mutual information*  for an alternative metric.

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mutual_info_score.html

In [25]:
# let's calculate Mutual Information for unigrams and bigrams
vectorizer = CountVectorizer(ngram_range=(1,1), stop_words=uk_stop_words, max_features=500)
countvector = vectorizer.fit_transform(reviews)
densevector = np.array(countvector.todense())
    
# miScore_unigram = pd.DataFrame(data = {'word': vectorizer.get_feature_names(),
#              'MI Score': [mutual_info_score(gtScore, densevector[:,i].squeeze()) for i in range(500)]})

miScore_unigram = pd.DataFrame(data =
                               {'MI Score': [mutual_info_score(gt_score, densevector[:,i].squeeze()) for i in range(500)]}
                              , index = vectorizer.get_feature_names())



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [44]:
# Bigram version
vectorizer = CountVectorizer(ngram_range=(2,2), stop_words=uk_stop_words, max_features=500)
countvector = vectorizer.fit_transform(reviews)
densevector = np.array(countvector.todense())
miScore_bigram = pd.DataFrame(data =
                    {'MI Score': [mutual_info_score(gt_score, densevector[:,i].squeeze()) for i in range(500)]},
                    index = vectorizer.get_feature_names())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [26]:
miScore_unigram.sort_values('MI Score', inplace=True, ascending=False)
print('Mutual Information - Unigram')
miScore_unigram.head(10)

Mutual Information - Unigram


,MI Score
персонал,0.067876
розташування,0.062621
чисто,0.040511
привітний,0.031284
nan,0.027092
зручне,0.019928
хороший,0.018283
місце,0.018195
чистий,0.017932
затишно,0.016047


In [30]:
miScore_bigram.sort_values('MI Score', inplace=True, ascending=False)
print('Mutual Information - Bigram')
miScore_bigram.head(10)

Mutual Information - Bigram


,MI Score
привітний персонал,0.033663
зручне розташування,0.017697
місце розташування,0.009666
приємний персонал,0.009136
погана звукоізоляція,0.007070
чудове розташування,0.006828
інтер єр,0.006487
центрі міста,0.006430
персонал привітний,0.006034
ванній кімнаті,0.005576


###  3. Pointwise Mutual Information

In [69]:
def getPMI_ngram(reviews, gt_score, gt, ngram_range=(1,1), max_features=500):
    vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words=uk_stop_words, max_features=max_features)
    countvector = vectorizer.fit_transform(reviews)
    densevector = np.array(countvector.todense())
    
    px = sum(gt_score == gt) / len(reviews)
    pmis = []
    
    for i in range(max_features):
        py = sum(densevector[:,i] == 1) / len(reviews)
        pxy = sum((gt_score== gt) & (densevector[:,i] == 1)) / len(reviews)
        
        if pxy == 0:
            pmi = math.log10((pxy + 0.0001) / (px * py))
        else:
            pmi = math.log10(pxy / (px * py))
            
        pmis.append(pmi)
        
    gt_name = 'positive' if gt == 1 else 'negative'
    pmis = pd.DataFrame(data = {'pmi' + gt_name: pmis}, index = vectorizer.get_feature_names())
    return pmis.sort_values('pmi' + gt_name, ascending=False)

In [70]:
gt_score = np.array(gt_score)

In [72]:
pmiPos_unigram = getPMI_ngram(reviews, gt_score,  1, max_features=2000)
pmiNeg_unigram = getPMI_ngram(reviews, gt_score,  0, max_features=2000)
pmiPos_bigram = getPMI_ngram(reviews, gt_score,  1, ngram_range=(2,2), max_features=2000)
pmiNeg_bigram = getPMI_ngram(reviews, gt_score,  0, ngram_range=(2,2), max_features=2000)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий'

### Let's see what PMI values tell us about the reviews

In [73]:
print('PMI for positive reviews - Unigram')
pmiPos_unigram.head(10)

PMI for positive reviews - Unigram


,pmipositive
торговий,0.251974
стильний,0.251648
просторо,0.250339
шикарне,0.249222
відмінне,0.249179
похвал,0.248875
необхідним,0.247499
чудове,0.247456
просторий,0.246833
швидке,0.246613


In [79]:
pmiPos_trigram[900:1000]

,score
ngram,


In [49]:
print('PMI for positive reviews - Bigram')
pmiPos_bigram.head(10)

PMI for positive reviews - Bigram


,pmipositive
чудовий сніданок,0.255065
розташування хороший,0.255065
чистота затишок,0.255065
тепло чисто,0.255065
усім необхідним,0.255065
персонал смачний,0.254037
номері необхідне,0.253878
просторий чистий,0.253786
торговий центр,0.253465
розташування приємний,0.253423


In [50]:
print('PMI for negative reviews - Unigram')
pmiNeg_unigram.head(10)

PMI for negative reviews - Unigram


,pminegative
погана,0.342608
відсутня,0.339392
неприємний,0.339246
брудні,0.338132
брудно,0.337215
тонкі,0.334506
брудний,0.334448
неможливо,0.332829
незручно,0.331634
жахливий,0.330902


In [55]:
print('PMI for negative reviews - Bigram')
pmiNeg_bigram.head(10)

PMI for negative reviews - Bigram


,pminegative
погана звукоізоляція,0.349638
погана шумоізоляція,0.347431
відсутність кондиціонера,0.346193
відбувається коридорі,0.343876
запах каналізації,0.343157
спати неможливо,0.342714
неприємний запах,0.342251
погано працював,0.342223
тонкі стіни,0.341177
чути відбувається,0.340378


In [80]:
pmiPos_trigram = getPMI_ngram(reviews, gt_score, 1, ngram_range=(3,3), max_features=2000)
pmiNeg_trigram = getPMI_ngram(reviews, gt_score, 0, ngram_range=(3,3), max_features=2000)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['дев', 'ласка', 'разу', 'ятий', 'ятнадцятий', 'ятнадцять', 'ять', 'ім'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [81]:
pmiPos_trigram

,pmipositive
персонал ввічливий готовий,0.255065
приємний персонал смачні,0.255065
приємний персонал хороший,0.255065
приємний персонал чистий,0.255065
персонал смачні ситні,0.255065
чисто тепло комфортно,0.255065
кухня усім необхідним,0.255065
чисто тихо комфортно,0.255065
приємний персонал чистота,0.255065
кімната зручне ліжко,0.255065


In [54]:
pmiNeg_trigram

,pminegative
запах ванній кімнаті,0.352442
неприємний запах ванній,0.352442
слабкий натиск води,0.352442
погана звукоізоляція чути,0.352442
запах каналізації номері,0.352442
погана шумоізоляція чути,0.352442
чутно відбувається коридорі,0.348246
погана звукоізоляція чутно,0.347303
чути відбувається коридорі,0.347241
слабкий wi fi,0.345493


In [40]:
df['pos_text'][137]

'Всі фотографії відповідають реальності. Шторка, робоча розетка, лампа. Зручне ліжко, новий посуд і чисті ванні кімнати.Ремонт новий. Адміністратори дуже приємні і виконують свою роботу швидко та якісно. Враження най-найкращі.'

## Save pmi results to file

In [82]:
pmiPos_bigram.index.name = 'ngram'
pmiPos_bigram = pmiPos_bigram.rename(columns={"pmipositive" : "score"})


In [83]:
pmiPos_bigram.to_csv('./data/bigram-pmi-positive-scores.csv')

In [84]:
pmiPos_unigram.index.name = 'ngram'
pmiPos_unigram = pmiPos_unigram.rename(columns={"pmipositive" : "score"})

In [85]:
pmiPos_unigram.to_csv('./data/unigram-pmi-positive-scores.csv')

In [86]:
pmiNeg_unigram.index.name = 'ngram'
pmiNeg_unigram = pmiNeg_unigram.rename(columns={"pminegative" : "score"})

In [87]:
pmiNeg_unigram.to_csv('./data/unigram-pmi-negative-scores.csv')

In [88]:
pmiNeg_bigram.index.name = 'ngram'
pmiNeg_bigram = pmiNeg_bigram.rename(columns={"pminegative" : "score"})

In [89]:
pmiNeg_bigram.to_csv('./data/bigram-pmi-negative-scores.csv')

In [92]:
pmiPos_trigram.index.name = 'ngram'
pmiPos_trigram = pmiPos_trigram.rename(columns={"pmipositive" : "score"})

In [93]:
pmiPos_trigram.to_csv('./data/trigram-pmi-positive-scores.csv')

In [94]:
pmiNeg_trigram.index.name = 'ngram'
pmiNeg_trigram = pmiNeg_trigram.rename(columns={"pminegative" : "score"})

In [95]:
pmiNeg_trigram.to_csv('./data/trigram-pmi-negative-scores.csv')

In [96]:
pmiNeg_trigram

,score
ngram,
відсутність питної води,0.352442
запах ванній кімнаті,0.352442
звукоізоляція залишає бажати,0.352442
порожній міні бар,0.352442
слабкий сигнал wi,0.352442
погана звукоізоляція номерів,0.352442
чути відбувається сусідніх,0.352442
відсутність одноразових капців,0.352442
відсутність міні бару,0.352442


In [101]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [125]:
display(pmiPos_unigram.head(5))
display(pmiPos_bigram.head(5))
display(pmiPos_trigram.head(5))

,score
ngram,
торговий,0.251974
стильний,0.251648
просторо,0.250339
шикарне,0.249222
відмінне,0.249179


,score
ngram,
місцезнаходження супер,0.255065
привітна господиня,0.255065
чудове співвідношення,0.255065
просторі кімнати,0.255065
хороші номери,0.255065


,score
ngram,
персонал ввічливий готовий,0.255065
приємний персонал смачні,0.255065
приємний персонал хороший,0.255065
приємний персонал чистий,0.255065
персонал смачні ситні,0.255065


In [120]:
pmiPos_bigram = pmiPos_bigram.drop(pmiPos_bigram.index[2])

In [105]:
pmiPos_unigram.columns

Index(['score'], dtype='object')

In [104]:
pmiPos_unigram.rename(columns={'ngram': 'unigram'})

,score
ngram,
торговий,0.251974
стильний,0.251648
просторо,0.250339
шикарне,0.249222
відмінне,0.249179
похвал,0.248875
необхідним,0.247499
чудове,0.247456
просторий,0.246833
